# Audio tiempo  Real en Julia y NLD

La idea es implementar un integrador de ODEs que produzca audio en tiempo real con control interactivo de parámetros en Julia.

Tomamos como base: 

https://github.com/JuliaAudio/PortAudio.jl

https://github.com/JuliaAudio/SampledSignals.jl

y algunos ejemplos de:

https://github.com/rob-luke/AuditoryStimuli.jl

https://www.theoj.org/joss-papers/joss.03613/10.21105.joss.03613.pdf


In [8]:
using Printf, DSP, Unitful

In [7]:
using Pkg; Pkg.activate("../../../NonLinearDynamicsCourse")

In [9]:
using PortAudio

Portaudio nos da acceso a la plataforma de audio. 
- PortAudio.devices() nos da la lista de dispositivos (muchos con APIs repetidas en windows) podemos indexar el array de devices
- PortAudioStream(devicename, inchans, outchans) abre un stream de audio para el dispositivo (sink para las salidas y source para las entradas)


In [10]:
sdev = PortAudio.devices()

26-element Vector{PortAudio.PortAudioDevice}:
 "Microsoft Sound Mapper - Input" 2→0
 "Microphone (Logitech BRIO)" 2→0
 "Microphone (Realtek(R) Audio)" 2→0
 "Microsoft Sound Mapper - Output" 0→2
 "Speakers (Realtek(R) Audio)" 0→8
 "Nx Headphones (Waves Virtual Au" 0→8
 "Realtek Digital Output (Realtek" 0→2
 "Primary Sound Capture Driver" 2→0
 "Microphone (Logitech BRIO)" 2→0
 "Microphone (Realtek(R) Audio)" 2→0
 "Primary Sound Driver" 0→2
 "Speakers (Realtek(R) Audio)" 0→8
 "Nx Headphones (Waves Virtual Audio Device)" 0→8
 "Realtek Digital Output (Realtek(R) Audio)" 0→2
 "Nx Headphones (Waves Virtual Audio Device)" 0→8
 "Speakers (Realtek(R) Audio)" 0→2
 "Realtek Digital Output (Realtek(R) Audio)" 0→2
 "Microphone (Realtek(R) Audio)" 2→0
 "Microphone (Logitech BRIO)" 2→0
 "Speakers (Realtek HD Audio output)" 0→8
 "Stereo Mix (Realtek HD Audio Stereo input)" 2→0
 "Line In (Realtek HD Audio Line input)" 2→0
 "Microphone (Realtek HD Audio Mic input)" 2→0
 "SPDIF Out (Realtek HDA SPDIF Out)

In [11]:
soundcard = PortAudioStream(sdev[5],0,2)

PortAudioStream{Float32}
  Samplerate: 44100.0Hz
  2 channel sink: "Speakers (Realtek(R) Audio)"

In [12]:
using SampledSignals

**SampledSignals** tiene el abstract type **SampleSource** que representa una fuente de samples (por ejemplo la función que integra las ODEs). 
La idea es implementar un subtipo que realice la integracion. Debe tener los metodos
- samplerate
- nchannels
- eltype
- unsafe_read!

**unsafe_read!** lee los samples de la fuente asumiendo que la cantidad de canales el sampling rate y el element type son acordes. Esta funcion es llamada por read! (o read) de forma externa una vez que verifica el acuerdo.


In [13]:
include("ToneSource.jl")

unsafe_read! (generic function with 1 method)

In [16]:
import Pkg; Pkg.add("Pipe")

In [17]:
using Pipe: @pipe

In [18]:
methods(ToneSource)

# 4 methods for type constructor:
[1] ToneSource(eltype, samplerate::Union{Quantity{T, 𝐓^-1, U}, Level{L, S, Quantity{T, 𝐓^-1, U}} where {L, S}} where {T, U}, freq::Number) in Main at C:\Users\Camilo\Google Drive\Julia\Audio\ToneSource.jl:10
[2] ToneSource(eltype, samplerate::Union{Quantity{T, 𝐓^-1, U}, Level{L, S, Quantity{T, 𝐓^-1, U}} where {L, S}} where {T, U}, freq::Number, phase::Number) in Main at C:\Users\Camilo\Google Drive\Julia\Audio\ToneSource.jl:10
[3] ToneSource(eltype, samplerate::Number, freq::Number) in Main at C:\Users\Camilo\Google Drive\Julia\Audio\ToneSource.jl:6
[4] ToneSource(eltype, samplerate::Number, freq::Number, phase::Number) in Main at C:\Users\Camilo\Google Drive\Julia\Audio\ToneSource.jl:6

In [19]:
methods(NoiseSource)

LoadError: UndefVarError: NoiseSource not defined

In [20]:
samplerate(source::ToneSource) = source.samplerate

samplerate (generic function with 1 method)

In [21]:
dump(SinusoidSource)

LoadError: UndefVarError: SinusoidSource not defined

In [26]:
dump(ToneSource)

UnionAll
  var: TypeVar
    name: Symbol T
    lb: Union{}
    ub: Any
  body: ToneSource{T} <: SampleSource
    samplerate::Float64
    freq::Float64
    phase::Float64


In [36]:
methods(nchannels)

# 3 methods for generic function "nchannels":
[1] nchannels(source::ToneSource) in Main at C:\Users\Camilo\Google Drive\Julia\Audio\ToneSource.jl:14
[2] nchannels(source::NoiseSource) in Main at C:\Users\Camilo\Google Drive\Julia\Audio\NoiseSource.jl:16
[3] nchannels(source::NoiseSource2) in Main at In[33]:16

In [23]:
tone_source = NoiseSource(Float64, 44100, 2, 0.3)

LoadError: UndefVarError: NoiseSource not defined

In [22]:
mutable struct NoiseSource2{T} <: SampleSource
    samplerate::Float64
    nchannels::Int64
    std::Float64

    function NoiseSource2(eltype, samplerate::Number, nchannels::Int, std::Number=1)
        new{eltype}(samplerate, nchannels, std)
    end
    function NoiseSource2(eltype, samplerate::Unitful.Frequency, nchannels::Int, std::Number=1)
        samplerate = ustrip(uconvert(u"Hz", samplerate))
        NoiseSource2(eltype, samplerate, nchannels, std)
    end
end

Base.eltype(::NoiseSource2{T}) where T = T
nchannels(source::NoiseSource2) = source.nchannels
samplerate(source::NoiseSource2) = source.samplerate

function unsafe_read!(source::NoiseSource2, buf::Array, frameoffset, framecount)
    buf[1+frameoffset:framecount+frameoffset, 1:source.nchannels] = source.std .* randn(framecount, source.nchannels)
    framecount
end

unsafe_read! (generic function with 2 methods)

In [53]:
using AuditoryStimuli

In [56]:
eltype(tone_source)

Float64

In [59]:
eltype(SinSource)

Any

In [58]:
methods(samplerate)

# 3 methods for generic function "samplerate":
[1] samplerate(source::NoiseSource) in Main at C:\Users\Camilo\Google Drive\Julia\Audio\NoiseSource.jl:17
[2] samplerate(source::NoiseSource2) in Main at In[33]:17
[3] samplerate(source::ToneSource) in Main at C:\Users\Camilo\Google Drive\Julia\Audio\ToneSource.jl:15

In [24]:
tone_source = SinSource(Float64, 44100, [200, 310])

SinSource{Float64}(44100.0, [0.02849517146113191, 0.04416751576475446], [0.0, 0.0])

In [25]:
for n = 1:30
    @pipe read(tone_source, 0.1u"s") |> write(soundcard, _)
end    

In [26]:
# Instansiate the audio stream in its own thread
noise_stream = Threads.@spawn begin
    while true
        @pipe read(tone_source, 1u"s") |> write(soundcard, _)
    end
end

Task (runnable) @0x000000000b862100

In [ ]:
methods(samplerate)

In [ ]:
soundcard = get_soundcard_stream("Speakers")

In [ ]:
noise_source = NoiseSource(Float64, 44100, 2, 0.2)
tone_source = SinusoidSource(Float64, 44100, 200)
amplify = Amplification(0.1, 0.01, 0.005)

In [ ]:
# Instansiate the audio stream in its own thread
noise_stream = Threads.@spawn begin
    while amplify.current > 0.001
        @pipe read(tone_source, 1u"s") |> modify(amplify, _) |> write(soundcard, _)
    end
end

In [ ]:
setproperty!(amplify, :target, 0)

In [ ]:
tone_source.freqs = [0.021]

In [30]:
noise_stream

Task (runnable) @0x000000000b862100

In [31]:
close(soundcard)

0

In [ ]:
noise_stream

In [32]:
# Real time ODE integration

In [33]:
using DifferentialEquations

LoadError: Failed to precompile DifferentialEquations [0c46a032-eb83-5123-abaf-570d42b7fbaa] to C:\Users\Camilo\.julia\compiled\v1.7\DifferentialEquations\jl_93FD.tmp.